#### **Student - Bălăcescu Bogdan**

In [4]:
import pandas as pd

df = pd.read_csv("hf://datasets/suayptalha/Poetry-Foundation-Poems/PoetryFoundationData.csv")

## Web Scraping Nichita Stanescu Poetry Titles

In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://www.romanianvoice.com/poezii/poeti/stanescu.php"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "ro-RO,ro;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive"
}

session = requests.Session()
session.headers.update(headers)

response = session.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

titles = soup.find_all("b")

titles = titles[:len(titles) - 1]

for title in titles:
    print(title.get_text(strip=True))


Argotice
Sensul iubirii
O viziune a sentimentelor
Dreptul la timp
Obiecte cosmice
Oul și sfera
Roșu vertical
Necuvintele
Un pământ numit România
În dulcele stil clasic
Belgradul în cinci prieteni
Măreția frigului
Epica magna
Operele imperfecte
Noduri și semne


## Subtitles

In [6]:
poems = []

for li in soup.find_all("li"):
    a = li.find("a")
    if a and a.get("href"):
        title = a.get_text(strip=True)
        href = a["href"]
        poems.append((href, title))


poems = list(dict(poems).items())

print(f"Found {len(poems)} poems\n")


Found 140 poems



## Poetry Content

In [7]:
from bs4 import BeautifulSoup, NavigableString, Tag
from urllib.parse import urljoin

poetry_dictionary = {}

for href, title in poems:
    poetry_url = urljoin(url, href)
    print(f"\nProcessing: {title}\n")

    response = session.get(poetry_url)
    response.raise_for_status()
    page_soup = BeautifulSoup(response.text, "html.parser")

    tds = page_soup.find_all("td")
    verses = max(tds, key=lambda td: len(td.get_text(strip=True)))

    container = verses.find("span", style=True) or verses

    lines = []

    for node in container.descendants:
        if isinstance(node, NavigableString):
            if node.parent and node.parent.name == "audio":
                continue

            text = node.strip()
            if text and not text.startswith("Audio"):
                lines.append(text)

        elif isinstance(node, Tag):
            if node.name == "br":
                lines.append("\n")
            elif node.name == "audio":
                continue

    poem_text = "".join(lines).strip()
    poetry_dictionary[title] = poem_text

    print(poem_text)
    print("-" * 60)



Processing: Balada motanului

Nichita StănescuBalada motanului

Motan m-aș fi dorit să fiu
cu coada-n sus, cu blana-n dungi,
cu gheare și mustețe lungi,
c-un ochi verzui și-un ochi căprui.

La ora când târâș-grăpiș
zăpada nopții se adună
eu, cocoțat pe-acoperiș,
să urlu a pustiu la lună.

Și-atuncea, șapte gospodine
să dea cu bolovani în mine
și să mă-njure surd, de Domnul,
că le-am stricat, urlând, tot somnul.

De sus, din vârful săptămânii,
să le rânjesc urlat, scârbos:
iubesc doar locul nu stăpânii,
precum fac câinii pentr-un os.

Și iarăși șapte gospodine
să dea cu bolovani în mine,
iar eu să urlu, urlu-ntruna
atât cât n-o apune luna.

Motan m-aș fi dorit să fiu
cu coada-n sus, cu blana-n dungi,
cu gheare și mustețe lungi
c-un ochi verzui și-un ochi căprui.

Când zorii ziua o deznoadă
să mă tot duc, să mă tot duc
și tinicheaua prinsă-n coadă
s-o zdrăngănesc pe străzi, năuc.

Jegos și obosit, apoi,
cu mațele în liturghie,
să mă adun, să mă-ncovoi
prin albiturile-n frânghie.

Ca-n f

In [8]:
for key, value in poetry_dictionary.items():
    lines = value.splitlines()
    poetry_dictionary[key] = "\n".join(
        line.strip() for index, line in enumerate(lines) if index > 0
    )

for key, value in poetry_dictionary.items():
    print(f"Title: {key}\n")
    print(value)
    print("-" * 60)



Title: Balada motanului


Motan m-aș fi dorit să fiu
cu coada-n sus, cu blana-n dungi,
cu gheare și mustețe lungi,
c-un ochi verzui și-un ochi căprui.

La ora când târâș-grăpiș
zăpada nopții se adună
eu, cocoțat pe-acoperiș,
să urlu a pustiu la lună.

Și-atuncea, șapte gospodine
să dea cu bolovani în mine
și să mă-njure surd, de Domnul,
că le-am stricat, urlând, tot somnul.

De sus, din vârful săptămânii,
să le rânjesc urlat, scârbos:
iubesc doar locul nu stăpânii,
precum fac câinii pentr-un os.

Și iarăși șapte gospodine
să dea cu bolovani în mine,
iar eu să urlu, urlu-ntruna
atât cât n-o apune luna.

Motan m-aș fi dorit să fiu
cu coada-n sus, cu blana-n dungi,
cu gheare și mustețe lungi
c-un ochi verzui și-un ochi căprui.

Când zorii ziua o deznoadă
să mă tot duc, să mă tot duc
și tinicheaua prinsă-n coadă
s-o zdrăngănesc pe străzi, năuc.

Jegos și obosit, apoi,
cu mațele în liturghie,
să mă adun, să mă-ncovoi
prin albiturile-n frânghie.

Ca-n fața unui șobolan
spinarea să mi-o fac c

In [9]:
import json

result_file = "results.jsonl"

with open(result_file, "w", encoding="utf-8") as f:
    for title, poem in poetry_dictionary.items():
        lines = poem.splitlines()

        verse_index = 1

        for i, line in enumerate(lines):
            line = line.strip()

            if not line:
                continue

            is_stanza_end = (
                i + 1 == len(lines) or        
                not lines[i + 1].strip()    
            )

            verse_text = line + (" @" if is_stanza_end else "")

            record = {
                "title": title,
                "verse_index": verse_index,
                "verse": verse_text
            }

            f.write(json.dumps(record, ensure_ascii=False) + "\n")
            verse_index += 1


In [ ]:
output_text = ""

with open(result_file, "r", encoding="utf-8") as f:
    lines = f.readlines()
    output_text += "START OF POEM\n\n"
    for line in lines:
        data = json.loads(line)
        if data["verse"][-1] == "@":
            output_text += data["verse"][:-1] + "\n\n"
        else:
            output_text += data["verse"] + "\n"

print(output_text)

Motan m-aș fi dorit să fiu
cu coada-n sus, cu blana-n dungi,
cu gheare și mustețe lungi,
c-un ochi verzui și-un ochi căprui. 

La ora când târâș-grăpiș
zăpada nopții se adună
eu, cocoțat pe-acoperiș,
să urlu a pustiu la lună. 

Și-atuncea, șapte gospodine
să dea cu bolovani în mine
și să mă-njure surd, de Domnul,
că le-am stricat, urlând, tot somnul. 

De sus, din vârful săptămânii,
să le rânjesc urlat, scârbos:
iubesc doar locul nu stăpânii,
precum fac câinii pentr-un os. 

Și iarăși șapte gospodine
să dea cu bolovani în mine,
iar eu să urlu, urlu-ntruna
atât cât n-o apune luna. 

Motan m-aș fi dorit să fiu
cu coada-n sus, cu blana-n dungi,
cu gheare și mustețe lungi
c-un ochi verzui și-un ochi căprui. 

Când zorii ziua o deznoadă
să mă tot duc, să mă tot duc
și tinicheaua prinsă-n coadă
s-o zdrăngănesc pe străzi, năuc. 

Jegos și obosit, apoi,
cu mațele în liturghie,
să mă adun, să mă-ncovoi
prin albiturile-n frânghie. 

Ca-n fața unui șobolan
spinarea să mi-o fac colan
să scuip, să 